# Satellite Imagery from Sentinel 2

Before running the notebook, do the following steps first (following instructions and code [here](https://github.com/artefactory/medium_satellite_imagery)):

*   create a geojson file of the zone you want a satellite imagery of and save it in 1_collection/city.geojson
*   create an account at Coppernicus Open Access Hub. Write the username and the password in a json file and save it in 1_collection/sentinel_api_credentials.json. Format: {"username": "XXXX", "password": "XXXX"}




## Step 0: Requirements & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install sentinelsat

In [ ]:
import os 
import cv2
from datetime import date
import json
import itertools
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd

## Step 1: Authentication with Copernicus Open Access Hub

In [ ]:
with open("./sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

## Step 2: Define area of interest in a geojson file

A way to specify the zones you want to explore is to provide the API with a geojson file, which is a json containing GPS coordinates position (latitude and longitude) of a zone. Geojson can be quickly created on this [website](https://geojson.io/#map=2/20/0).

In [ ]:
geojson_path = "./vinnytsia.geojson"

## Step 3: Query satellite images via Sentinelsat API

In [ ]:
shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2021, 1, 1), date(2022, 1, 30)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 5)
)

images_df = api.to_dataframe(images)

We can use the is_online() function from the sentinelsat package to filter the queried images that are available online. A for loop iterates over the images dictionary and checks if each image is online. 

In [ ]:
online_images = {}
for id, item in images.items():
    if api.is_online(id):
        online_images[id] = item

# Convert the filtered images to a dataframe
online_images_df = api.to_dataframe(online_images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
online_images_df = online_images_df.sort_values(by=['cloudcoverpercentage'], ascending=True)
online_images_df[key_columns].head(5)

,platformserialidentifier,processinglevel,cloudcoverpercentage,vegetationpercentage,beginposition
02bae0c5-d47f-4f00-94db-2bf7005196f7,Sentinel-2B,Level-2A,0.011257,49.927589,2021-05-10 09:05:49.024
447797aa-e70b-4105-8b5e-ce72b3b79149,Sentinel-2B,Level-2A,0.011370,95.476073,2021-07-09 09:05:59.024
d4a9ee7e-f8e5-43b6-b6e8-6edfab580f1b,Sentinel-2B,Level-2A,0.035212,94.082057,2021-07-09 09:05:59.024
296fbe58-3c6b-4c58-b970-050244a71fec,Sentinel-2B,Level-2A,0.078759,14.998293,2021-04-10 09:05:49.024
190b3f86-4b0f-448f-a1d9-051dbfdd9e40,Sentinel-2B,Level-2A,0.101761,15.382813,2021-04-10 09:05:49.024


##  Step 4: Download satellite image

In [ ]:
uuid = "190b3f86-4b0f-448f-a1d9-051dbfdd9e40"
api.download(uuid)

MD5 checksumming:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

{'id': '190b3f86-4b0f-448f-a1d9-051dbfdd9e40',
 'title': 'S2B_MSIL2A_20210410T090549_N0300_R050_T35UPQ_20210410T120243',
 'size': 1107918856,
 'md5': 'e30166889940cb75d7a77f8518c987ba',
 'date': datetime.datetime(2021, 4, 10, 9, 5, 49, 24000),
 'footprint': 'POLYGON((28.3851737331767 49.64443670244057,29.90457276290215 49.61627372192671,29.847478451946774 48.62982157570405,28.35791076985194 48.657027178567795,28.3851737331767 49.64443670244057))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('190b3f86-4b0f-448f-a1d9-051dbfdd9e40')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2021, 4, 10, 16, 2, 34, 980000),
 'Ingestion Date': datetime.datetime(2021, 4, 10, 16, 2, 34, 980000),
 'manifest_name': 'manifest.safe',
 'product_root_dir': 'S2B_MSIL2A_20210410T090549_N0300_R050_T35UPQ_20210410T120243.SAFE',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('190b3f86-4b0f-448f-a1d9-051dbfdd9e40')/Products('Quicklook')/$value",
 'path': 'S2B_M

## Step 5: Unzip satellite image and save to folder

In [ ]:
!unzip /content/S2B_MSIL2A_20210410T090549_N0300_R050_T35UPQ_20210410T120243.zip -d ../data/sentinel2/vinnytsia

# Credits

*   Antoine Aubay (2021): Leveraging satellite imagery for machine learning computer vision applications. Available at: https://medium.com/artefact-engineering-and-data-science/leveraging-satellite-imagery-for-machine-learning-computer-vision-applications-d22143f72d94

